In [1]:
import os
from models.midi_transformer import MIDITransformer
from models.helpers.blocks import TransformerXLEncoderStack, TransformerXLDecoderStack
import tensorflow as tf

In [2]:
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians"
DATA_DIR = os.path.join(BASE_DIR, 'data', 'POP909-Dataset')
MODEL_SAVE_PATH = os.path.join(BASE_DIR, 'checkpoints', 'model1')
MODEL_CONFIG_PATH = os.path.join(BASE_DIR, 'model-store', 'models', 'configs', 'default.json')

In [3]:
#transformer = MIDITransformer(MODEL_CONFIG_PATH, MODEL_SAVE_PATH, dynamic=True)

configs = {
	"encoder": {
        "num_layers": "4",
        "d_model": "128",
        "num_heads": "8",
        "dff": "128",
        "input_vocab_size": "256",
        "maximum_position_encoding": "256",
        "memory_length": "256",
        "dropout_rate": "0.1",
        "max_sequence_length": "256"
	},
	"decoder": {
        "num_layers": "4",
        "d_model": "128",
        "num_heads": "8",
        "dff": "128",
        "target_vocab_size": "256",
        "maximum_position_encoding": "128",
        "memory_length": "256",
        "dropout_rate": "0.1",
        "max_sequence_length": "128"
	}
}

encoder = TransformerXLEncoderStack(configs['encoder'], dynamic=True)
decoder = TransformerXLDecoderStack(configs['decoder'], dynamic=True)

In [4]:
inputx = tf.random.uniform(shape=(10,256), maxval=128, dtype=tf.int32)
pe = tf.random.uniform(shape=(512,128), dtype=tf.float32)
look_ahead_mask = tf.sequence_mask([200], maxlen=256)
look_ahead_mask = tf.tile(tf.expand_dims(look_ahead_mask, axis=1), [1, 10, 1])

encoder_output = encoder([inputx, tf.constant(256)], pe)
print(tf.shape(encoder_output))
decoder_output, attn_weights = decoder([inputx, tf.constant(256)], encoder_output, pe, look_ahead_mask)
print(tf.shape(decoder_output))
print(tf.shape(attn_weights))

tf.Tensor([ 10 256 128], shape=(3,), dtype=int32)
tf.Tensor([ 10 256 128], shape=(3,), dtype=int32)
tf.Tensor([ 10 256 128], shape=(3,), dtype=int32)
